# Input

In [1]:
# we are using the google api to fetch us data
from langchain.chat_models import ChatOpenAI
from pyspark_ai import SparkAI

# If 'gpt-4' is unavailable, use 'gpt-3.5-turbo' (might lower output quality)
llm = ChatOpenAI(model_name='gpt-4', temperature=0)

# Initialize SparkAI with the ChatOpenAI model
spark_ai = SparkAI(llm=llm, verbose=True)

# Activate partial functions for Spark DataFrame
spark_ai.activate()

23/07/05 12:23:05 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp4s0)
23/07/05 12:23:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 12:23:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Search and ingest content from url into a DataFrame
tech_comp_df = spark_ai.create_df("https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/", 
                                  ["company_name", "market_cap_trillion", "country"])
tech_comp_df.show()

INFO: Parsing URL: https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW tech_companies AS 
SELECT * FROM VALUES
('Apple', 3.027, 'USA'),
('Microsoft', 2.513, 'USA'),
('Alphabet (Google)', 1.525, 'USA'),
('Amazon', 1.336, 'USA'),
('NVIDIA', 1.047, 'USA'),
('Tesla', 0.88688, 'USA'),
('Meta Platforms (Facebook)', 0.73299, 'USA'),
('TSMC', 0.53497, 'Taiwan'),
('Tencent', 0.41274, 'China'),
('Samsung', 0.37211, 'S. Korea')
AS v1(company_name, market_cap_trillion, country)

INFO: Storing data into temp view: tech_companies

+--------------------+-------------------+--------+
|        company_name|market_cap_trillion| country|
+--------------------+-------------------+--------+
|               Apple|            3.02700|     USA|
|           Microsoft|            2.51300|     USA|
|   Alphabet (Google)|            1.52500|     USA|
|              Amazon|            1.33600|     USA|
|              NVIDIA|     

In [21]:
# other ways to create a DataFrame
# By default it will create a dataframe with significant columns. We can specify the column names as well.
# tech_comp_df = spark_ai.create_df("https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/", 
#                                   ["company_name", "market_cap_trillion", "country"])

# We can also use google search query to get the data. We need GOOGLE_API_KEY and GOOGLE_CSE_ID for that.
tech_comp_df = spark_ai.create_df("largest tech companies by market cap", 
                                ["company_name", "market_cap_trillion", "country"])

INFO: Parsing URL: https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW tech_companies AS 
SELECT * FROM VALUES
('Apple', 3.027, 'USA'),
('Microsoft', 2.513, 'USA'),
('Alphabet (Google)', 1.525, 'USA'),
('Amazon', 1.336, 'USA'),
('NVIDIA', 1.047, 'USA'),
('Tesla', 0.88688, 'USA'),
('Meta Platforms (Facebook)', 0.73299, 'USA'),
('TSMC', 0.53497, 'Taiwan'),
('Tencent', 0.41274, 'China'),
('Samsung', 0.37211, 'S. Korea')
AS v1(company_name, market_cap_trillion, country)

INFO: Storing data into temp view: tech_companies



In [3]:
# we also have a local csv file that we can use 
# (https://github.com/HariVM/Analytics/blob/master/1000%20Sales%20Records.csv)
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tech_comp").getOrCreate()

manual_df = spark.read.csv("1000 Sales Records.csv", header=True, inferSchema=True)
manual_df.show()

23/07/05 12:24:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|   Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Middle East and N...|     Libya|      Cosmetics|      Offline|             M|10/18/2014|686800706|10/31/2014|      8446|     437.2|   263.33|    3692591.2|2224085.18|  1468506.02|
|       North America|    Canada|     Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|      3018|    154.06|    90.93|    464953.08| 274426.74|   190526.34|
|Middle East and N...|     Libya|      Baby Food|      Offline|             C|10/31/2016|246222

# Plotting

In [4]:
tech_comp_df.ai.plot()

INFO: Here is a Python code snippet that uses Plotly to visualize the data in the `df` DataFrame. This code assumes that you want to create a bar chart where the x-axis represents the `company_name` and the y-axis represents the `market_cap_trillion`. The bars are colored based on the `country`.


```
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Create a bar chart
fig = px.bar(pandas_df, x='company_name', y='market_cap_trillion', color='country', title='Market Cap of Tech Companies')

# Display the plot
fig.show()
```

This code first converts the Spark DataFrame to a Pandas DataFrame using the `toPandas()` method. Then, it uses the `plotly.express.bar()` function to create a bar chart. The `x`, `y`, and `color` parameters of the `bar()` function are set to 'company_name', 'market_cap_trillion', and 'country', respectively. The `title` parameter is set to 'Market Cap of Tech Companies'. Finally, the `show()`

In [5]:
tech_comp_df.ai.plot("company with market cap greater than 1 trillion")

INFO: Here is a Python code snippet that uses Plotly to visualize the result of `df`. This code assumes that `df` is a PySpark DataFrame. It first filters the DataFrame to include only companies with a market cap greater than 1 trillion, then converts the DataFrame to a Pandas DataFrame, and finally creates a bar plot with company names on the x-axis and market cap on the y-axis.


```
import plotly.express as px
from pyspark.sql.functions import col

# Filter the DataFrame to include only companies with a market cap greater than 1 trillion
df_filtered = df.filter(col("market_cap_trillion") > 1)

# Convert the PySpark DataFrame to a Pandas DataFrame
df_pandas = df_filtered.toPandas()

# Create a bar plot with company names on the x-axis and market cap on the y-axis
fig = px.bar(df_pandas, x='company_name', y='market_cap_trillion', color='country', title='Companies with Market Cap Greater Than 1 Trillion')

# Display the plot
fig.show()
```

This code uses the `filter` method of the PyS

In [6]:
# using manually created dataframe
manual_df.ai.plot("Sales count by Region")

INFO: Here is a Python code snippet that uses PySpark and Plotly to visualize the sales count by region. This code assumes that the PySpark DataFrame `df` is already defined and populated.


```
import plotly.express as px
from pyspark.sql import functions as F

# First, we need to aggregate the data by region and count the sales
df_agg = df.groupBy("Region").agg(F.count("Units Sold").alias("Sales Count"))

# Convert the aggregated Spark DataFrame to a Pandas DataFrame
df_pandas = df_agg.toPandas()

# Create a bar plot using Plotly
fig = px.bar(df_pandas, x='Region', y='Sales Count', title='Sales Count by Region')

# Display the plot
fig.show()
```

This code first groups the data by the 'Region' column and counts the number of sales in each region. The result is a new DataFrame `df_agg`. This DataFrame is then converted to a Pandas DataFrame, which is used to create a bar plot with Plotly. The plot is displayed directly in the notebook.


# Transformations

In [7]:
tech_comp_df.ai.transform("Name of the country with max companies and how much is the total market cap for those companies").show()

INFO: SQL query for the transform:
SELECT country, SUM(market_cap_trillion) as total_market_cap
FROM temp_view_for_transform
GROUP BY country
ORDER BY COUNT(company_name) DESC, total_market_cap DESC
LIMIT 1

+-------+----------------+
|country|total_market_cap|
+-------+----------------+
|    USA|        11.06787|
+-------+----------------+



In [8]:
tech_comp_df.ai.transform("list all the countries").show()

INFO: SQL query for the transform:
SELECT DISTINCT country FROM temp_view_for_transform

+--------+
| country|
+--------+
|     USA|
|  Taiwan|
|   China|
|S. Korea|
+--------+



In [9]:
manual_df.ai.transform("Sales sum for units sold by Region").show()

INFO: SQL query for the transform:
SELECT Region, SUM(`Units Sold`) as Sales_Sum
FROM temp_view_for_transform
GROUP BY Region

+--------------------+---------+
|              Region|Sales_Sum|
+--------------------+---------+
|Middle East and N...|   682363|
|Australia and Oce...|   417298|
|              Europe|  1285808|
|  Sub-Saharan Africa|  1386894|
|Central America a...|   503362|
|       North America|   100739|
|                Asia|   677524|
+--------------------+---------+



In [10]:
manual_df.ai.transform("Sum of all the orders served in North America").show()

INFO: SQL query for the transform:
SELECT SUM(`Units Sold`) as Total_Orders_Served
FROM temp_view_for_transform
WHERE Region = 'North America'

+-------------------+
|Total_Orders_Served|
+-------------------+
|             100739|
+-------------------+



# Explain

In [11]:
tech_comp_df.ai.explain()

'In summary, this dataframe is creating a view named `tech_companies` from a local relation `v1`. The view contains three columns: `company_name`, `market_cap_trillion`, and `country`. The `company_name` and `country` columns are cast to string type, and the `market_cap_trillion` column is cast to decimal type with precision 6 and scale 5.'

In [12]:
manual_df.ai.explain()

'In summary, this dataframe is retrieving all records from the CSV file. The columns include Region, Country, Item Type, Sales Channel, Order Priority, Order Date, Order ID, Ship Date, Units Sold, Unit Price, Unit Cost, Total Revenue, Total Cost, and Total Profit.'

# Verify

In [13]:
tech_comp_df.ai.verify("name of companies should be unique")

INFO: Generated code:
def has_unique_company_names(df) -> bool:
    from pyspark.sql import functions as F

    # Count the number of unique company names
    unique_company_names = df.select(F.countDistinct("company_name")).collect()[0][0]

    # Check if the number of unique company names is equal to the total number of rows
    if unique_company_names == df.count():
        return True
    else:
        return False

result = has_unique_company_names(df)

INFO: 
Result: True


In [4]:
manual_df.ai.verify("Sales count by Region should be greater than 1000")

INFO: Generated code:
def check_sales_count(df) -> bool:
    from pyspark.sql.functions import col, sum

    # Group by Region and sum the Units Sold
    df_grouped = df.groupBy("Region").agg(sum("Units Sold").alias("Total Sales"))

    # Check if the minimum Total Sales is greater than 1000
    min_sales = df_grouped.agg({"Total Sales": "min"}).collect()[0][0]

    if min_sales > 1000:
        return True
    else:
        return False

result = check_sales_count(df)

INFO: 
Result: True


# UDFs

In [15]:
# one of the examples
@spark_ai.udf
def convert_grades(grade_percent: float) -> str:
    """Convert the grade percent to a letter grade using standard cutoffs"""

INFO: Creating following Python UDF:
def convert_grades(grade_percent) -> str:
    if grade_percent is not None:
        if grade_percent >= 90.0:
            return 'A'
        elif grade_percent >= 80.0:
            return 'B'
        elif grade_percent >= 70.0:
            return 'C'
        elif grade_percent >= 60.0:
            return 'D'
        else:
            return 'F'



In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.udf.register("convert_grades", convert_grades)
percentGrades = [(1, 97.8), (2, 72.3), (3, 81.2)]
df = spark.createDataFrame(percentGrades, ["student_id", "grade_percent"])
df.selectExpr("student_id", "convert_grades(grade_percent)").show()

+----------+-----------------------------+
|student_id|convert_grades(grade_percent)|
+----------+-----------------------------+
|         1|                            A|
|         2|                            C|
|         3|                            B|
+----------+-----------------------------+



In [17]:
@spark_ai.udf
def diff_bw_ship_date_and_order_date(ship_date, order_date) -> int:
    """Order date and ship date are date strings formatted as MM/DD/YYYY, calculate the difference in days"""

INFO: Creating following Python UDF:
def diff_bw_ship_date_and_order_date(ship_date, order_date):
    from datetime import datetime
    if ship_date is not None and order_date is not None:
        ship_date = datetime.strptime(ship_date, '%m/%d/%Y')
        order_date = datetime.strptime(order_date, '%m/%d/%Y')
        return (ship_date - order_date).days



In [18]:
spark.udf.register("diff_bw_ship_date_and_order_date", diff_bw_ship_date_and_order_date)
manual_df.createOrReplaceTempView("manualDF")

In [19]:
manual_df.ai.transform("difference between ship date and order date using udf diff_bw_ship_date_and_order_date ").show()

INFO: SQL query for the transform:
SELECT 
    `Order ID`, 
    `Order Date`, 
    `Ship Date`, 
    diff_bw_ship_date_and_order_date(`Order Date`, `Ship Date`) as Date_Difference
FROM 
    temp_view_for_transform

+---------+----------+----------+---------------+
| Order ID|Order Date| Ship Date|Date_Difference|
+---------+----------+----------+---------------+
|686800706|10/18/2014|10/31/2014|            -13|
|185941302| 11/7/2011| 12/8/2011|            -31|
|246222341|10/31/2016| 12/9/2016|            -39|
|161442649| 4/10/2010| 5/12/2010|            -32|
|645713555| 8/16/2011| 8/31/2011|            -15|
|683458888|11/24/2014|12/28/2014|            -34|
|679414975|  3/4/2015| 4/17/2015|            -44|
|208630645| 5/17/2012| 6/28/2012|            -42|
|266467225| 1/29/2015|  3/7/2015|            -37|
|118598544|12/24/2013| 1/19/2014|            -26|
|451010930|12/29/2015| 1/19/2016|            -21|
|220003211| 2/27/2010| 3/18/2010|            -19|
|702186715|11/17/2016|12/22/2016|  

# Output

In [20]:
spark_ai.commit()